In [1]:
import cv2 as cv
import numpy as np

In [2]:
def apply_DM_1(arr, Fact):
    n = len(arr)
    new_arr = np.zeros(n, dtype=np.uint8)
    i = 0
    while i < n:
        # find the first zero value
        if arr[i] != 0:
            new_arr[i] = arr[i]
            i += 1
        else:
            # find the next non-zero value
            j = i+1
            while j < n and arr[j] == 0:
                j += 1
            if j == n:
                # if there are no non-zero values after the current zero,
                # use the previous non-zero value
                new_arr[i:] = new_arr[i-1]
            else:
                # interpolate between the previous non-zero value and the next non-zero value
                Max = max(arr[i-1], arr[j])
                Min = min(arr[i-1], arr[j])
                diff = j - i
                for k in range(i, j):
                    new_arr[k] = round(((Max - Min) / Fact) * ((k-i) ) + Min)
            i = j
    return new_arr


In [3]:
def apply_DM_1(arr, Fact):
    n = len(arr)
    new_arr = np.zeros(n, dtype=np.int16)
    i = 0
    li = []
    for i in range(len(arr)):
        if i*Fact < len(arr):
            li.append(i*Fact)
        else:
            break

    # Apply DM_1 to each pair of adjacent indices in the li list
    for i in range(len(li)-1):
        start = li[i]
        if len(li) == i+1:
            end = 0
            slope = (arr[start] // 2) 
        else :    
            end = li[i+1]
            slope = ((arr[end] // 2) - (arr[start] // 2))

        new_arr[start] = arr[start]
        new_arr[end] = arr[end]
        for j in range(start, end):
            new_arr[j] = abs(round((arr[start] // 2) + slope * (j - start)) * 2)

    return new_arr


In [4]:
def apply_DM_1(arr, Fact):
    li = np.arange(0,len(arr),Fact)

    for i in range(1,len(li)):
        start = li[i-1]
        end = li[i]
        slope = (arr[end] - arr[start])
        for i in range(start+1, end):
            arr[i] = round( (slope/Fact) * ( i - start ) )+ arr[start]
            
    if int(len(li)) % 2 == 1:
        last_pixel = li[-1]
        arr[last_pixel:] = [arr[last_pixel]]*len(arr[last_pixel:])
    return arr


[10, 7, 3, 0, 10, 20, 30]


In [6]:
def direct_1_order_gray(img, factor):
    # Get the dimensions of the input image
    height, width = img.shape

    # Compute the new dimensions of the resized image
    new_height, new_width = height * factor, width * factor

    # Create a new image of zeros with the new dimensions
    new_img = np.zeros((new_height, new_width))

    # Move the pixels from the old image to the new image
    for i in range(height):
        for j in range(width):
            new_img[i * factor, j * factor] = img[i, j]

    # Iterate over each row in the new image and interpolate the values between consecutive zeros
    for i in range(new_height):
        new_img[i] = apply_DM_1(new_img[i], factor)

    # Transpose the new image to iterate over the columns
    new_img_T = new_img.T

    # Iterate over each column in the new image and interpolate the values between consecutive zeros
    for i in range(new_width):
        new_img_T[i] = apply_DM_1(new_img_T[i], factor)

    # Transpose the new image back to its original orientation
    new_img = new_img_T.T

    return new_img.astype('uint8') 


In [7]:
def direct_1_order_rgb(img, factor):
    # Get the dimensions of the input image
    height, width, channels = img.shape

    # Compute the new dimensions of the resized image
    new_height, new_width = height * factor, width * factor

    # Create a new image of zeros with the new dimensions
    new_img = np.zeros((new_height, new_width, channels))

    # Move the pixels from the old image to the new image
    for i in range(height):
        for j in range(width):
            new_img[i * factor, j * factor, :] = img[i, j, :]

    # Iterate over each channel in the new image
    for channel in range(channels):
        # Iterate over each row in the new image
        for i in range(new_height):
            new_img[i, :, channel] = apply_DM_1(new_img[i, :, channel], factor)

        # Iterate over each column in the new image
        for j in range(new_width):
            new_img[:, j, channel] = apply_DM_1(new_img[:, j, channel], factor)

    return new_img.astype('uint8') 


In [10]:
img=cv.imread('image/1.jpeg')
factor = 2
new_img = direct_1_order_rgb(img, factor)
print(new_img)
cv.imshow('Orginal image',img)
cv.imshow(f'Resize image X{factor}',new_img)
cv.waitKey(0)


[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]


-1